In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import os
import glob

In [2]:
raw_dat_dir = '/home/grace/Documents/python/acute_oral/data/raw/'
processed_dat_dir = '/home/grace/Documents/python/acute_oral/data/processed/'
interim_dat_dir = '/home/grace/Documents/python/acute_oral/data/interim/'
figures_dir = '/home/grace/Documents/python/acute_oral/reports/figures/'
test_dir = '/home/grace/Documents/python/acute_oral/data/external/TEST/'
times_dir = '/home/grace/Documents/python/acute_oral/data/external/TIMES/'

In [3]:
xls = pd.ExcelFile(processed_dat_dir+'relevant_acute_pred_final_200320.xlsx')

In [4]:
xls.sheet_names

['relevant_pred']

In [5]:
relevant_acute_data = xls.parse('relevant_pred')

In [6]:
relevant_acute_data.shape

(8979, 32)

In [7]:
relevant_acute_data[(relevant_acute_data['TEST_LD50_mgkg'].isnull()) & (~relevant_acute_data['TEST_pLD50'].isnull()) ]

,Unnamed: 0,DTXSID,CAS,name,known_LD50_mgkg,known_pLD50,TEST_LD50_mgkg,TEST_pLD50,TIMES_LD50_mgkg,TIMES_pLD50,...,molecular_initiating_event_3,effect_biological outcome_3,alert_reliability,parameter_ranges,correct_fragments_structural_domain,incorrect_fragments_structural_domain,unknown_fragments_structural_domain,total_structural_domain,total_domain,acute_oral_toxicity_confidence
3667,3667,NaN,29387-86-8,NaN,1900.0,NaN,NaN,1.29,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4639,4639,NaN,39699-08-6,NaN,29000.0,NaN,NaN,1.30,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
chemotypes = pd.read_csv(raw_dat_dir+'Acute_SMILES_Toxprints.tsv', sep = '\t')

In [9]:
chemotypes.rename(columns = {'M_NAME': 'CAS'}, inplace = True)

In [10]:
for_enrich = relevant_acute_data[['DTXSID', 'CAS', 'name', 'known_LD50_mgkg', 'known_pLD50',
       'TEST_LD50_mgkg', 'TEST_pLD50', 'TIMES_LD50_mgkg', 'TIMES_pLD50',
       'TIMES', 'TEST', 'known_pLD50_minus_TEST', 'known_pLD50_minus_TIMES',
       'average_mass', 'qsar_ready_smiles', 'smiles']]

In [11]:
for_enrich.shape

(8979, 16)

In [12]:
for_enrich_txps = pd.merge(for_enrich, chemotypes, on = 'CAS', how = 'left')

In [13]:
TEST_for_enrich = for_enrich_txps[(~for_enrich_txps['TEST'].isnull())]

In [14]:
TIMES_for_enrich = for_enrich_txps[(~for_enrich_txps['TIMES'].isnull())]

In [15]:
TEST_for_enrich['TEST_out_CI'] = TEST_for_enrich['TEST'].apply(lambda x: 0 if x == 'Within_CI' else 1)

/home/grace/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [16]:
TIMES_for_enrich['TIMES_out_CI'] = TIMES_for_enrich['TIMES'].apply(lambda x: 0 if x == 'Within_CI' else 1)

/home/grace/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [17]:
TIMES_for_enrich = TIMES_for_enrich.set_index('DTXSID')
TEST_for_enrich = TEST_for_enrich.set_index('DTXSID')

In [18]:
TIMES_for_enrich.columns.get_loc('atom:element_main_group')
TEST_for_enrich.columns.get_loc('atom:element_main_group')

15

In [19]:
import scipy.stats as stats

In [31]:
res_TEST = []
for i, column in TEST_for_enrich.iloc[:,15:-1].iteritems():
    if pd.crosstab(TEST_for_enrich.iloc[:,744],column).shape == (2,2):
        a = pd.crosstab(TEST_for_enrich.iloc[:,744],column)
        tn, fp, fn, tp = a.iloc[0,0], a.iloc[0,1], a.iloc[1,0], a.iloc[1,1]
        tested_active = (tp + fn)
        oddsratio, pvalue = stats.fisher_exact(a)
        res_TEST.append([oddsratio, pvalue, i, tp, fn, fp, tn])


In [32]:
res_TIMES = []
for i, column in TIMES_for_enrich.iloc[:,15:-1].iteritems():
    if pd.crosstab(TIMES_for_enrich.iloc[:,744],column).shape == (2,2):
        a = pd.crosstab(TIMES_for_enrich.iloc[:,744],column)
        tn, fp, fn, tp = a.iloc[0,0], a.iloc[0,1], a.iloc[1,0], a.iloc[1,1]
        tested_active = (tp + fn)
        oddsratio, pvalue = stats.fisher_exact(a)
        res_TIMES.append([oddsratio, pvalue, i, tp, fn, fp, tn])

In [33]:
def convert(lst):
    df = pd.DataFrame(lst)
    df.rename(columns = {0: 'OR', 1: 'p', 2 : 'TxP', 3: 'TP', 4: 'FN', 5: 'FP', 6: 'TN'}, inplace = True)
    df = df.sort_values(by=['OR','p'], ascending = [True, False])
    return df

In [34]:
TEST_enrichments = convert(res_TEST)
TIMES_enrichments = convert(res_TIMES)

In [35]:
pd.crosstab(TEST_for_enrich.iloc[:,744],TEST_for_enrich.loc[:,'bond:CC(=O)C_ketone_alkene_cyclic_2-en-1-one'])

bond:CC(=O)C_ketone_alkene_cyclic_2-en-1-one,0.0,1.0
TEST_out_CI,,
0,581,7
1,1000,31


In [36]:
TEST_enrichments[(TEST_enrichments['OR'] >= 2.5) & (TEST_enrichments['p'] < 0.05 ) & (TEST_enrichments['TP'] >=3)]

,OR,p,TxP,TP,FN,FP,TN
54,2.573000,0.025145,bond:CC(=O)C_ketone_alkene_cyclic_2-en-1-one,31,1000,7,581
207,4.251734,0.010707,bond:S(=O)N_sulfonamide,22,1009,3,585
226,4.375624,0.001863,bond:S~N_generic,30,1001,4,584
208,5.044776,0.002795,bond:S(=O)N_sulfonylamide,26,1005,3,585
434,12.204950,0.001140,ring:hetero_[6_6]_O_benzopyrone_(1_4-),21,1010,1,587


In [37]:
TIMES_enrichments[(TIMES_enrichments['OR'] >= 2.5) & (TIMES_enrichments['p'] < 0.05 ) & (TIMES_enrichments['TP'] >=3)]

,OR,p,TxP,TP,FN,FP,TN
144,3.280142,0.006806,bond:quatN_alkyl_acyclic,30,282,6,185
146,3.523810,0.002877,bond:quatN_generic,32,280,6,185
147,4.063709,0.022398,bond:quatN_trimethyl_alkyl_acyclic,19,293,3,188
240,6.943522,0.035363,group:ligand_path_5-7_bidentate,11,301,1,190
68,inf,0.026908,bond:COH_alcohol_allyl,8,304,0,191
65,inf,0.015381,bond:COC_ether_alkenyl,9,303,0,191


In [38]:
#TIMES_for_enrich
mask = (TEST_for_enrich['bond:S(=O)N_sulfonamide'] == 1) | (TEST_for_enrich['bond:S(=O)N_sulfonamide'] == 1)

In [39]:
specific_txps = TEST_for_enrich[mask]

In [40]:
specific_txps.columns 

test_cols = ['CAS', 'name', 'known_LD50_mgkg', 'known_pLD50', 'TEST_LD50_mgkg',
       'TEST_pLD50', 'TEST']

In [41]:
specific_txps[specific_txps['TEST_out_CI'] ==1][test_cols].to_csv(processed_dat_dir+'examples_TESTenrichedTxPs.csv')

In [42]:
mask2 = (TIMES_for_enrich['bond:COH_alcohol_allyl']==1)| (TIMES_for_enrich['bond:COC_ether_alkenyl']==1)

In [43]:
inf_txps = TIMES_for_enrich[mask2]

In [44]:
inf_txps[['CAS', 'name', 'known_LD50_mgkg', 'TIMES_LD50_mgkg',
       'TIMES']].to_csv(processed_dat_dir+'examples_TIMESenrichedTxPs.csv')